In [1]:
#importing module
from langchain_community.tools import DuckDuckGoSearchRun
#dependency need to be install first pip install -U duckduckgo-search

# using Some Built in Tools in langchain

In [2]:
#creating an object of DuckDuckGoSearchRun class.
search_engine = DuckDuckGoSearchRun()
answer = search_engine.invoke(input = "effect of tariff war in india ")
answer

c:\anaconda\envs\hf-gpu\lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


'3 days ago · A 50 percent tariff on India will impact trade worth billions of dollars and risk Washington-New Delhi ties. 2 days ago · Trump India Tariffs Impact LIVE News Updates: The 50% US tariff, which is one … Aug 7, 2025 · US Tariff on India: Trump imposed an additional 25% Tariff on Indian goods. … Tariff news highlights: The additional 25% tariffs on Indian imports, imposed by US President … Aug 7, 2025 · The US has imposed a sweeping 50% tariff on Indian exports, pushing US-India …'

# 3 ways to create custom tools in langchain futhure we can add this tool to llm to take action again defined task

# way-1:  by using decorator tools we can create 

In [3]:
from langchain_core.tools import tool


#tools in langchain is nothing but user defined function.

@tool
def multiplication(a:int,b:int) ->int:
    """multipliying two numbers"""
    return a*b

In [4]:
#As we know that tools is a runnable means futhure tool can  be used in chains,so that we can invoke the tool .
result = multiplication.invoke(input={'a':3,'b':2})
result


6

In [5]:
# if i want to show the schema of above tools that can also will be seen by LLM Model.
print(multiplication.args_schema.model_json_schema())

{'description': 'multipliying two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiplication', 'type': 'object'}


In [6]:
print(multiplication.name)
print(multiplication.description)
print(multiplication.args)

multiplication
multipliying two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


# way-2 : to create tools by using SturctureTools(using pydantic it achieve) class

In [7]:
from langchain.tools import StructuredTool
from pydantic import BaseModel,Field
from typing import Annotated,Literal,TypedDict



#creating pydantic class to validate the types of field
class MultiplyingFactor(BaseModel):
    a : Annotated[int,Field(...,description="provided first integer variable to multiply")]
    b : Annotated[int,Field(...,description="provided second integer variable to multiply")]

In [8]:
def multiplication(a:int,b:int) ->int:
    """multipliying two numbers"""
    return a*b


#now creating an object of Sturcture toll class
structure = StructuredTool.from_function(
    func=multiplication,
    name="multiplying two numbers",
    args_schema=MultiplyingFactor
)

In [9]:
#calling the tool
result = structure.invoke(input={'a':2,'b':3})
result

6

In [10]:
# Let's inspect some of the attributes associated with the tool.
print(structure.name)
print(structure.description)
print(structure.args)
print(structure.return_direct)

multiplying two numbers
multipliying two numbers
{'a': {'description': 'provided first integer variable to multiply', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'provided second integer variable to multiply', 'title': 'B', 'type': 'integer'}}
False


# way:3 By using BaseTool class of Langchain we can create Custom Tool

In [11]:
from langchain.tools import BaseTool
from pydantic import BaseModel,Field
from typing import Annotated,Literal,TypedDict,Optional


In [12]:
#creating pydantic class to validate the types of field
class MultiplyingFactor(BaseModel):
    a : Annotated[int,Field(...,description="provided first integer variable to multiply")]
    b : Annotated[int,Field(...,description="provided second integer variable to multiply")]

# below we are creating custom class by inheriting property from BaseTool class

In [13]:
class Multiplication1(BaseTool):
    #now defining the meta data about this custom tool class
    name : str = "multiplication of two numbers"
    description :str = "useful for when you need to answer questions about math"
    args_schema : type[BaseModel]  = MultiplyingFactor
    
    
    #using magic method to define mthod of this class
    def _run(self,a,b ):
        return a*b
        
    

In [14]:
#creating an object of this class
multi = Multiplication1()
result = multi.invoke(input={'a':2,'b':3})
result

6

In [15]:
# to see the meta data abou this tools
print(multi.name)
print(multi.description)
print(multi.args)
print(multi.return_direct)

multiplication of two numbers
useful for when you need to answer questions about math
{'a': {'description': 'provided first integer variable to multiply', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'provided second integer variable to multiply', 'title': 'B', 'type': 'integer'}}
False
